In [2]:
import datamol as dm 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from umap import UMAP
from scipy.cluster import hierarchy
import random 
import torch
from random import shuffle
from cluster_margin.cluster_margin_dummy import ClusterMargin
import pytorch_lightning as pl 


%load_ext autoreload
%autoreload 2


In [3]:
from cluster_margin.utils import get_dummy_data

In [18]:
data = get_dummy_data()
data.head()

,smiles,mol,fp,x,y,prob0,prob1,prob2,prob3,prob4
0,Br.CC(N)Cc1ccc(O)cc1,<rdkit.Chem.rdchem.Mol object at 0x17f273140>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.486177,5.127035,0.117005,0.241573,0.119706,0.197060,0.324655
1,Br.CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1,<rdkit.Chem.rdchem.Mol object at 0x17f2725e0>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9.875013,4.257200,0.046248,0.324113,0.203771,0.169230,0.256638
2,Br.CC1C2Cc3ccc(O)cc3C1(C)CCN2CCc1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x17f2710e0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.827803,4.044406,0.162232,0.372045,0.187925,0.208073,0.069724
3,Br.CCN(c1cc(-c2ccc(CN3CCOCC3)cc2)cc(C(=O)NCc2c...,<rdkit.Chem.rdchem.Mol object at 0x17f270040>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9.965529,4.780724,0.275930,0.262057,0.035593,0.292248,0.134172
4,Br.CN(C)CCCC1(c2ccc(F)cc2)OCc2cc(C#N)ccc21,<rdkit.Chem.rdchem.Mol object at 0x17f273ae0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",10.303421,4.024837,0.106179,0.099227,0.328980,0.211405,0.254209


In [43]:
cm=ClusterMargin(data, distance_threshold=0.5, iterations=10, km=100, k_batch = 5, verbose = True) 

2023-09-18 09:57:53.952 | INFO     | cluster_margin.cluster_margin:seed:84 - Setting seed to 42


In [20]:
cm.predict_proba([10,4,2])

tensor([[0.2499, 0.2592, 0.1085, 0.2718, 0.1106],
        [0.2205, 0.0942, 0.2912, 0.3452, 0.0489],
        [0.2522, 0.1602, 0.2346, 0.1532, 0.2000]])

In [28]:
cm._first_iteration=True
cm._do_iteration()

2023-09-18 09:46:02.735 | INFO     | cluster_margin.cluster_margin:_initialize_embedding:102 - Initializing embedding, selected idxs : [1166  457  291 1398 1702  980  904 1407  605  937 1467 1927  410  954
  408 1532  892 1604    5  336 1115  478 1917 1224 1822 1015  214 1785
 1746 1281]
2023-09-18 09:46:02.759 | INFO     | cluster_margin.cluster_margin:get_M:119 - Predicted probabilities for unlabelled idxs tensor([[0.3899, 0.2949, 0.0846, 0.1269, 0.1037],
        [0.1318, 0.1735, 0.0774, 0.4338, 0.1834],
        [0.3567, 0.1589, 0.1181, 0.2167, 0.1496],
        ...,
        [0.2837, 0.0461, 0.4362, 0.2141, 0.0200],
        [0.1394, 0.2491, 0.0582, 0.1826, 0.3708],
        [0.1370, 0.3192, 0.1732, 0.1616, 0.2089]]) 


[1713, 1895, 587, 961, 489]

In [23]:
len(cm.labelled_idxs)

30

In [45]:

cm.query()

2023-09-18 09:58:09.511 | INFO     | cluster_margin.cluster_margin:_do_iteration:169 - Starting iteration 1
2023-09-18 09:58:09.512 | INFO     | cluster_margin.cluster_margin:predict_proba:114 - Predicting probabilities for idxs [   0    1    2 ... 1932 1933 1934]
2023-09-18 09:58:09.513 | INFO     | cluster_margin.cluster_margin:get_M:121 - Predicted probabilities for unlabelled idxs tensor([[0.1643, 0.1963, 0.0977, 0.2532, 0.2885],
        [0.2901, 0.0154, 0.1552, 0.2670, 0.2722],
        [0.1096, 0.2454, 0.2578, 0.1383, 0.2489],
        ...,
        [0.1888, 0.2527, 0.1783, 0.1368, 0.2435],
        [0.0424, 0.3736, 0.1926, 0.0187, 0.3727],
        [0.2201, 0.2133, 0.1494, 0.1843, 0.2329]]) 
2023-09-18 09:58:09.514 | INFO     | cluster_margin.cluster_margin:get_M:131 - Querying idxs [ 856 1929 1484 1339 1426  711  792  626  192 1755  348 1667  297  716
  813  982 1847 1858  645   88 1772 1245  733 1539 1506  745  764  493
 1933 1461 1565 1352 1503 1000  696 1074 1429 1630  160 1356  

[614, 978, 462, 1755, 1266]

In [36]:
cm.query()

2023-09-18 09:50:01.956 | INFO     | cluster_margin.cluster_margin:get_M:119 - Predicted probabilities for unlabelled idxs tensor([[0.3420, 0.0044, 0.0971, 0.0584, 0.4980],
        [0.2446, 0.1090, 0.2635, 0.3779, 0.0050],
        [0.1222, 0.4513, 0.3044, 0.0460, 0.0762],
        ...,
        [0.0845, 0.2947, 0.1726, 0.1330, 0.3151],
        [0.2317, 0.0978, 0.3202, 0.1953, 0.1550],
        [0.3096, 0.3366, 0.0106, 0.3208, 0.0223]]) 
2023-09-18 09:50:01.957 | INFO     | cluster_margin.cluster_margin:update:159 - Updating labelled idxs with [302, 585, 375, 481, 660]
2023-09-18 09:50:01.958 | INFO     | cluster_margin.cluster_margin:update:161 - New len labelled idxs 50


[302, 585, 375, 481, 660]

In [37]:
cm.labelled_idxs

array([1541, 1828, 1408, 1694,  416,  112,  399, 1160,  144, 1406,  623,
        975, 1622, 1240,  466,  685,  123,  951, 1729,  592,  509, 1286,
       1130,  735,   53,   95,  344, 1011,   21,  851,   93, 1655, 1321,
       1789,  292, 1202, 1801, 1439,  875,  333,  786, 1768,  722,  462,
        636,  302,  585,  375,  481,  660])

In [38]:
cm=ClusterMargin(data, distance_threshold=0.5, iterations=10, km=100, k_batch = 5) 

2023-09-18 09:52:33.853 | INFO     | cluster_margin.cluster_margin:seed:83 - Setting seed to 42


In [46]:
for i in cm.query_iterator():
    print(i)

2023-09-18 09:58:43.094 | INFO     | cluster_margin.cluster_margin:_do_iteration:169 - Starting iteration 2
2023-09-18 09:58:43.096 | INFO     | cluster_margin.cluster_margin:predict_proba:114 - Predicting probabilities for idxs [   0    1    2 ... 1932 1933 1934]
2023-09-18 09:58:43.098 | INFO     | cluster_margin.cluster_margin:get_M:121 - Predicted probabilities for unlabelled idxs tensor([[0.0030, 0.2477, 0.1882, 0.2286, 0.3325],
        [0.2432, 0.1252, 0.1098, 0.2699, 0.2521],
        [0.3089, 0.2440, 0.3037, 0.1117, 0.0317],
        ...,
        [0.2816, 0.3081, 0.1837, 0.0721, 0.1544],
        [0.3186, 0.2624, 0.0984, 0.0512, 0.2693],
        [0.2283, 0.2261, 0.0971, 0.2184, 0.2301]]) 
2023-09-18 09:58:43.100 | INFO     | cluster_margin.cluster_margin:get_M:131 - Querying idxs [ 696  968   56 1514 1671  163 1835  323  137  209 1415 1828 1163 1257
  661 1301 1568  157 1133  592  486  379   80  816  459 1151 1578 1010
  313  655 1122  134 1843  675  128 1491 1331 1459 1934  221 1

[1934, 708, 1465, 701, 655, 221, 137, 128, 1459, 1142]
[1896, 16, 289, 1382, 1236, 162, 760, 40, 1045, 914]
[1826, 1078, 1479, 649, 1754, 1690, 798, 870, 1724, 1811]
[134, 722, 1224, 1362, 1053, 430, 1894, 459, 681, 497]
[1627, 1630, 765, 597, 868, 1535, 1128, 1393, 1892, 22]
[313, 1655, 1751, 501, 598, 1689, 741, 554, 78, 905]
[1193, 540, 789, 155, 1797, 1218, 1570, 1487, 30, 247]
[242, 1920, 1225, 73, 1895, 1292, 1493, 936, 1235, 1688]


In [41]:
len(cm.labelled_idxs)

130

In [42]:
for i in cm: 
    print(i)